### Heroes Of Pymoli Data Analysis
* Of the 1163 active players, the vast majority are male (84%). There also exists, a smaller, but notable proportion of female players (14%).

* Our peak age demographic falls between 20-24 (44.8%) with secondary groups falling between 15-19 (18.60%) and 25-29 (13.4%).  
-----

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)
purchase_data.head()


,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


## Player Count

* Display the total number of players


In [3]:
#get rid of unneeded columns and repeat data
df_unique = purchase_data[["SN","Gender", "Age"]].drop_duplicates()
df_unique.set_index(["SN"],drop=True, inplace=True)

#total players
totalPlayers = df_unique["Gender"].count()

df_total = pd.DataFrame(columns=['Total Players'])
df_total.loc[0] = totalPlayers
df_total

,Total Players
0,576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [4]:
#get values
unique_items = purchase_data.groupby("Item ID")["Item ID"].nunique().count()
average_price =purchase_data["Price"].mean() 
number_purchased = purchase_data["Purchase ID"].count()
total_revenue = purchase_data["Price"].sum()

#create summary dataframe
df_sum = pd.DataFrame({"Number of Unique Items": [unique_items],"Average Price":[average_price], "Number of Purchases":number_purchased
                         ,"Total Revenue": total_revenue})
#format money values
df_sum["Average Price"] = df_sum["Average Price"].map("${:.2f}".format)
df_sum["Total Revenue"] = df_sum["Total Revenue"].map("${:.2f}".format)

#print
df_sum

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,$2379.77


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [5]:
#count of disctinct data entries
df_gender= df_unique.groupby(["Gender"]).count()

#name column
df_gender.columns = ["Total Count"]

#percentage
df_gender["Percentage of Players"] = (df_gender["Total Count"]/df_gender["Total Count"].sum())*100

#format percentage
df_gender["Percentage of Players"] = df_gender["Percentage of Players"].map("{0:.02f}%".format)

#print
df_gender


,Total Count,Percentage of Players
Gender,,
Female,81,14.06%
Male,484,84.03%
Other / Non-Disclosed,11,1.91%



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [7]:
#group aggregates
df_gen_group = purchase_data.groupby(["Gender","SN"]).agg(
    {'Price':['sum','count'],
     'Age':['mean']})

#drop top level index
df_gen_group.columns = df_gen_group.columns.droplevel(0)

#rename columns
df_gen_group.columns=["Price","Count","Age"]

#initial group aggregates
df_gen_sum = df_gen_group.groupby(["Gender"]).agg(
    {'Price':['mean','sum'],
     'Count':['sum'],
     'Age':['mean']})

#drop top level index
df_gen_sum.columns = df_gen_sum.columns.droplevel(0)

#rename columns
df_gen_sum.columns=["Avg Total Purchase per Person","Total Purchase Value","Total Purchases","Average Age"]

#create average purchase price column and add to dataframe
df_gen_sum["Average Purchase Price"] = df_gen_sum["Total Purchase Value"]/df_gen_sum["Total Purchases"]

#format values
df_gen_sum["Average Purchase Price"] = df_gen_sum["Average Purchase Price"].map("${:.2f}".format)
df_gen_sum["Avg Total Purchase per Person"] = df_gen_sum["Avg Total Purchase per Person"].map("${:.2f}".format)
df_gen_sum["Total Purchase Value"] = df_gen_sum["Total Purchase Value"].map("${:.2f}".format)

#reorder columns
df_gen_sum= df_gen_sum[["Total Purchases","Average Purchase Price","Total Purchase Value","Avg Total Purchase per Person"]]

#print
df_gen_sum.head()

,Total Purchases,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,$1967.64,$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [9]:
#bins and labels
bins = [0, 9, 14, 19, 24, 29,34,39,200]
bin_labels = ['<10', '10-14', '15-19', '20-24', '25-29', '30-34','35-40','40+']

#new column in df_unique
df_unique["Age Group"] =  pd.cut(df_unique["Age"], bins, labels=bin_labels)

#age group count
df_AgeGroup = df_unique.groupby("Age Group").count()

#rename column
df_AgeGroup = df_AgeGroup[["Gender"]].rename(columns={'Gender':'Count'})

#percentage
df_AgeGroup["Percentage"] = (df_AgeGroup["Count"]/df_AgeGroup["Count"].sum())*100

#format percentage
df_AgeGroup["Percentage"] = df_AgeGroup["Percentage"].map("{0:.02f}%".format)

#print
df_AgeGroup

,Count,Percentage
Age Group,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-40,31,5.38%
40+,12,2.08%


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [10]:
#get group aggregates
df_grouped = purchase_data.groupby(["SN"]).agg(
    {'Price':['count','sum']})

#drop first level index
df_grouped.columns = df_grouped.columns.droplevel(0)

#rename columns
df_grouped.columns=["Purchase Count","Total Purchase Value"]

#join df_unique and df_grouped to get one df
df_purchAge = df_unique.join(df_grouped,how="left")
df_purchAge

#new df groupby by Age group
df_AgeGrp = df_purchAge.groupby("Age Group")

#get sum
df_AgeGrp_sum = df_AgeGrp.sum()

#join dataframes
df_AgeFinal = df_AgeGroup.join(df_AgeGrp_sum, how = "left")

#average values
df_AgeFinal["Avg Total Purchase per Person"] = df_AgeFinal["Total Purchase Value"]/df_AgeFinal["Count"]
df_AgeFinal["Average Purchase Price"] = df_AgeFinal["Total Purchase Value"]/df_AgeFinal["Purchase Count"]

#clean unnecessary columns
df_AgeFinal = df_AgeFinal[["Purchase Count","Average Purchase Price","Total Purchase Value","Avg Total Purchase per Person"]]

#format values
df_AgeFinal["Average Purchase Price"] = df_AgeFinal["Average Purchase Price"].map("${:.2f}".format)
df_AgeFinal["Total Purchase Value"] = df_AgeFinal["Total Purchase Value"].map("${:.2f}".format)
df_AgeFinal["Avg Total Purchase per Person"] = df_AgeFinal["Avg Total Purchase per Person"].map("${:.2f}".format)

#print
df_AgeFinal


,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Group,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,$1114.06,$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-40,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [12]:
#get df with groupby SN from original purchase_data
df_players_group = purchase_data.groupby("SN")

#get sum
df_top_spenders = df_unique.join(df_players_group["Price"].sum())

#get count
df_top_spenders = df_top_spenders.join(df_players_group["Item ID"].count())

#average 
df_top_spenders["Average Purchase Price"] = df_top_spenders["Price"]/df_top_spenders["Item ID"]

#clean unnecessary columns and reorder columns
df_top_spenders = df_top_spenders[["Item ID","Average Purchase Price","Price"]]

#rename columns
df_top_spenders.rename(columns={"Item ID":"Purchase Count","Price":"Total Purchase Value"},inplace=True)

#sort by total purchase
df_top_spenders.sort_values("Total Purchase Value",ascending =False, inplace = True)  

#format values
df_top_spenders["Average Purchase Price"] = df_top_spenders["Average Purchase Price"].map("${:.2f}".format)
df_top_spenders["Total Purchase Value"] = df_top_spenders["Total Purchase Value"].map("${:.2f}".format)


df_top_spenders.head()


,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [17]:
#get group aggregates
df_item_group = purchase_data.groupby(["Item ID","Item Name"]).agg(
    {'Price':['count','mean','sum']})

#drop top level index
df_item_group.columns = df_item_group.columns.droplevel(0)

#rename columns
df_item_group.columns=["Purchase Count","Item Price","Total Purchase Value"]

#sort by total purchase
df_item_group1 = df_item_group.sort_values("Purchase Count",ascending =False)  

#format values
df_item_group1["Item Price"] = df_item_group1["Item Price"].map("${:.2f}".format)
df_item_group1["Total Purchase Value"] = df_item_group1["Total Purchase Value"].map("${:.2f}".format)

#print
df_item_group1.head(10)

The history saving thread hit an unexpected error (OperationalError('disk I/O error',)).History will not be written to the database.


,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16
103,Singed Scalpel,8,$4.35,$34.80
75,Brutality Ivory Warmace,8,$2.42,$19.36
72,Winter's Bite,8,$3.77,$30.16
60,Wolf,8,$3.54,$28.32


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [18]:
#sort by total purchase
df_item_group2 = df_item_group.sort_values("Total Purchase Value",ascending =False)  

#format values
df_item_group2["Item Price"] = df_item_group2["Item Price"].map("${:.2f}".format)
df_item_group2["Total Purchase Value"] = df_item_group2["Total Purchase Value"].map("${:.2f}".format)

df_item_group2.head(10)

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80
59,"Lightning, Etcher of the King",8,$4.23,$33.84
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
78,"Glimmer, Ender of the Moon",7,$4.40,$30.80
72,Winter's Bite,8,$3.77,$30.16


# Conclusion

The first trend that I noticed from the statistical data is, of the 576 total players, approximately 84% (484) of them are male.  Because of this large percentage of male players, males also contribute more in total purchases (652 to 113 for females) and in total value of purchases ($1967.64 to $361.94).  Women and Other genders did however spend more on each purchase ($3.20 to $3.02) as well as on average of total purchase ($4.47 to $4.07).

One of the major trends in the data became noticeable when analyzing the age demographics.  When broken down into brackets for every 5 years starting at <10, most of the players appeared concentrated in 3 age groups, 15-19 (18.58%), 20-24 (44.79%), and 25-29 (13.37%).  This means that these 3 age groups make up approximately 76.7% of all players.  In addidtion to this concentration, the most popular age group (20-24) contained more than the other two popular age groups combined (258 to 184).  This one age group alone makes up almost half of the percantage of total players (44.8%).  In accordance with those demographics, those age groups also spent more on Total Purchase value than any other age group ($1114.06).  When comparing expenditures between the 20-24 age group and the other two most popular age groups (15-19 and 25-29), members of the 20-24 group also spent more on each purchase and signicantly more then them on Average Total purchase per person ($4.32 to $3.86 and $3.81).

In terms of the items purchased, the data here seems pretty standardly distributed, there doesn't seem to be any major outliers or noticeably obvious trends, such as the male to female ratio of players or the percentage of total players in a certain age range.  There was one item however, the Oathbreaker, Last Hope of the Breaking storm.  This item sold more often (12 times) and for the greatest total value of sales ($50.76).  The only trend I noticed when examining all the data as a whole was that the more epxensive items also tend to sell the best.  Three of the top four selling items in terms of quantity also have an item price of more than $4.00